In [1]:
# RDD와 비교하기
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('spark_sql_basic')
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/07 02:02:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# RDD만을 이용한 데이터 추출

In [2]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

마블 영화 중 관객 수가 500만 이상인 영화 가져오기
1. Inner Join -> Filter By Movies -> Filter By Attendance
2. Filter By Movies, Attendance -> Inner Join

In [3]:
# Case 1. Join 먼저, filter 나중에
movie_att = movies_rdd.join(attendances_rdd)
movie_att.collect()

[(2, (('슈퍼맨', 'DC'), (2182227, 'KR'))),
 (4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (3, (('배트맨', 'DC'), (4226242, 'KR'))),
 (5, (('아이언맨', '마블'), (4300365, 'KR')))]

In [4]:
movie_att.filter(
    lambda x: x[1][0][1] == '마블' and x[1][1][0] >= 5000000).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [5]:
# Case 2. Filter 먼저, Join 나중에
movie

NameError: name 'movie' is not defined

In [7]:
filtered_mov=movies_rdd.filter(lambda x : x[1][1]=="마블")
filtered_att=attendances_rdd.filter(lambda x : x[1][0] >= 5000000)

filtered_mov.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

# Spark SQL 사용하기

SparkSession 생성
- SparkContext에 해당하며, 새로운 스파크 어플리케이션을 생성

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName("spark-sql").getOrCreate() # spark 이름이 바뀌면 안된다.
spark                                                              # 같은 이름이 있으면 가져오고 없으면 생성

RDD를 이용해서 데이터프레임 생성하기

In [9]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

movie_schema = ["id", "name", "company", "year", "month", "day"] # 형태는 들억가지 않았음.

## 데이터프레임 생성
- inferSchema 옵션을 활용하면 타입을 스파크가 자동으로 결정(기본)

In [10]:
df= spark.createDataFrame(data=movies, schema = movie_schema)

In [11]:
# 스키마 확인
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [12]:
# dataframe은 곧 rdd
df

DataFrame[id: bigint, name: string, company: string, year: bigint, month: bigint, day: bigint]

전체 데이터프레임의 내용을 확인
- show()

In [13]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



## 데이터프레임에 SQL 사용하기
- 데이터프레임을 temporary view에 등록해야 Spark SQL을 사용할 수 있다.

In [14]:
df.createOrReplaceTempView("movies") # movies라는 임시 테이블 생성

movies 테이블에서 영화 이름(name)만 가져오기

In [15]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- company: string (nullable = true)
 |-- year: long (nullable = true)
 |-- month: long (nullable = true)
 |-- day: long (nullable = true)



In [16]:
query = '''
SELECT name
FROM movies
'''

# query 실행
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [17]:
# 2010년 이후에 개봉한 영화를 조회
query='''
SELECT *
FROM movies
WHERE year>=2010
'''

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [18]:
# 제목이 ~~맨으로 끝나는 영화 정보 조회

In [19]:
query='''
SELECT *
FROM movies
WHERE name LIKE "%맨" 
'''
# __은 특정 개수를 채워야하기 때문에 ㄴㄴ 

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [20]:
# 개봉 월이 4~8월 사이인 영화 정보 조회
query='''
SELECT *
FROM movies
WHERE month BETWEEN 4 AND 8  
'''

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [21]:
# 회사 이름이 "마"로 시작하거나 "니"로 끝나는 영화 중 2010년 이후로 개봉한 영화
query='''
SELECT *
FROM movies
WHERE (company like "마%" OR company like "%니") and year >= 2010  
'''
# in에서는 %를 못쓴다.

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [22]:
# 개봉연도 오름차순 정렬
query='''
SELECT *
FROM movies
ORDER BY year ASC;  
'''

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [23]:
# 회사 별 몇개의 영화가 있는지 조회
query='''
SELECT company, count(year) as movie_cnt
FROM movies
GROUP BY company
'''

spark.sql(query).show()

+-------+---------+
|company|movie_cnt|
+-------+---------+
| 디즈니|        1|
|   마블|        2|
|     DC|        2|
+-------+---------+



In [24]:
attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [25]:
#직접 스키마 지정하기, 기존의 RDD를 테이블화시키기 위해 스키마를 정해주는 것
from pyspark.sql.types import StringType, FloatType, IntegerType # 데이터 타입

# StructField : 각 컬럼을 만들기 위한 집합, 필드 하나의 값, 컬럼스키마 작성, 이름,Type,....
# StructType : 컬럼 집합, 컬럼을 모두 포함한 인스턴스 정의
from pyspark.sql.types import StructType, StructField # 구조

In [26]:
att_schema = StructType([
    StructField("id",IntegerType(),True), # True는 null허용을 뜻한다.
    StructField("att",FloatType(),True),
    StructField("country",StringType(),True)
])

In [27]:
att_df = spark.createDataFrame(
    data = attendances,
    schema = att_schema
)

att_df.dtypes

[('id', 'int'), ('att', 'float'), ('country', 'string')]

In [28]:
att_df.createOrReplaceTempView("att")

In [29]:
att_df.show()

+---+-----------+-------+
| id|        att|country|
+---+-----------+-------+
|  1|1.3934592E7|     KR|
|  2|  2182227.0|     KR|
|  3|  4226242.0|     KR|
|  4|1.0303058E7|     KR|
|  5|  4300365.0|     KR|
+---+-----------+-------+



In [30]:
# 영화 아이디, 영화 이름, 회사, 관객수 조회
query = '''
SELECT movies.id, movies.name,movies.company, att.att
FROM movies
JOIN att ON movies.id = att.id
'''

spark.sql(query).show()

+---+--------+-------+-----------+
| id|    name|company|        att|
+---+--------+-------+-----------+
|  1|어벤져스|   마블|1.3934592E7|
|  2|  슈퍼맨|     DC|  2182227.0|
|  3|  배트맨|     DC|  4226242.0|
|  4|겨울왕국| 디즈니|1.0303058E7|
|  5|아이언맨|   마블|  4300365.0|
+---+--------+-------+-----------+



# 데이터프레임 API

In [33]:
# collect를 사용하면 RDD처럼 등장
df.select("*").collect()

[Row(id=1, name='어벤져스', company='마블', year=2012, month=4, day=26),
 Row(id=2, name='슈퍼맨', company='DC', year=2013, month=6, day=13),
 Row(id=3, name='배트맨', company='DC', year=2008, month=8, day=6),
 Row(id=4, name='겨울왕국', company='디즈니', year=2014, month=1, day=16),
 Row(id=5, name='아이언맨', company='마블', year=2008, month=4, day=30)]

In [34]:
df.select("name","company").collect()

[Row(name='어벤져스', company='마블'),
 Row(name='슈퍼맨', company='DC'),
 Row(name='배트맨', company='DC'),
 Row(name='겨울왕국', company='디즈니'),
 Row(name='아이언맨', company='마블')]

In [38]:
df.select(df.name, (df.year-2000).alias("year")).show() # 데이터를 가지고 온 뒤에 수정
# alias는 고치고자 하는 컬럼 뒤에

+--------+----+
|    name|year|
+--------+----+
|어벤져스|  12|
|  슈퍼맨|  13|
|  배트맨|   8|
|겨울왕국|  14|
|아이언맨|   8|
+--------+----+



In [36]:
df.select(df.name, (df.year-2000)).collect()

[Row(name='어벤져스', (year - 2000)=12),
 Row(name='슈퍼맨', (year - 2000)=13),
 Row(name='배트맨', (year - 2000)=8),
 Row(name='겨울왕국', (year - 2000)=14),
 Row(name='아이언맨', (year - 2000)=8)]

In [39]:
# agg : Aggregate의 약자. 그룹핑 후 데이터를 하나로 합쳐주는 역할
df.agg({"id":"count"}).collect()

[Row(count(id)=5)]

In [40]:
# query에서 사용가능한 함수들을 모아놓은 패키지가 존재
from pyspark.sql import functions as F

df.agg(F.min(df.year))

DataFrame[min(year): bigint]

In [41]:
# groupBy
df.groupBy().avg().collect() # 컬럼명이 부여되지 않으면 각 컬럼별로 집계를 수행한다.

[Row(avg(id)=3.0, avg(year)=2011.0, avg(month)=4.6, avg(day)=18.2)]

In [45]:
# 회사 별 월 별 영화 개수 정ㅂㅂ보
df.groupBy([df.company,df.month]).count().collect()
# df.groupBy([df.company,df.month]).agg({"id":"count"}).collect()

[Row(company='디즈니', month=1, count(id)=1),
 Row(company='DC', month=8, count(id)=1),
 Row(company='DC', month=6, count(id)=1),
 Row(company='마블', month=4, count(id)=2)]

In [46]:
# 회사 별 개봉 월 평균
df.groupBy("company").agg({"month":"mean"}).collect()

[Row(company='디즈니', avg(month)=1.0),
 Row(company='마블', avg(month)=4.0),
 Row(company='DC', avg(month)=7.0)]

In [47]:
# join
df.join(att_df,'id').select(df.name,att_df.att).show()

+--------+-----------+
|    name|        att|
+--------+-----------+
|어벤져스|1.3934592E7|
|  슈퍼맨|  2182227.0|
|  배트맨|  4226242.0|
|겨울왕국|1.0303058E7|
|아이언맨|  4300365.0|
+--------+-----------+



In [48]:
# select, where, orderBy 사용
df.select("name","company","year").where("company='마블'").orderBy("id").collect()

[Row(name='어벤져스', company='마블', year=2012),
 Row(name='아이언맨', company='마블', year=2008)]

In [49]:
spark.stop()
sc.stop()